# EXPLANATION

3. Regarding the topic-modeling, I’ll give you my current notebook for how I’m doing it right now, on OpenAlex data. You should be able to plug what you download right in:

https://drive.google.com/drive/folders/1e-AWJbWbhL-XivUT1buA0ELk1dKkgXAm?usp=sharing

This also includes an example of a map that might come out of this.



Some comments: The notebook is a bit messy. I’m afraid, I don’t have time to clean it up right now, but wanted to share the most current thing. The workflow is: a) Embed texts with a language model (for scientific articles I’m using Specter 2, for other stuff I would use mpnet or nomic-embed, for short texts USE (https://www.sbert.net/ is amazing for all of this!). I wouldn’t use Llama, you probably want an encoder, not a decoder model. b) Layout with UMAP for visualization c) Cluster directly on the embeddings with EvōC. EvōC is Leland McInnes’s bleeding edge multilayer clustering-algorithm. Really good, but awful to get running on a mac. You might have more luck in colab, or just use hDBSCAN on UMAP directly (I provide that code as well). d) Label the nested clusters with Llama 3 70B. You can do this manually, as well or use a smaller LLM, depending on your GPU-options. You should be able to use everything that works with Llama.ccp e) Plot the outcome with datamapplot (I’ve attached an .html file to the drive of what the final results look like).

In [ ]:
import os
from pathlib import Path
import sys

# --- ENVIRONMENT SWITCH ---
# Set to True if running on local machine with Google Drive Desktop mounted
# Set to False if running in Google Colab cloud
RUNNING_LOCALLY = False

if RUNNING_LOCALLY:
  # --- REPO ROOT ON sys.path (so `from src.*` works locally) ---
    _REPO_ROOT = str(Path(os.getcwd()).resolve().parents[1])
    if _REPO_ROOT not in sys.path:
        sys.path.insert(0, _REPO_ROOT)
    # Standard macOS path for Google Drive Desktop
    BASE_PATH = Path('/Volumes/GoogleDrive/My Drive/Colab Projects/AI Public Trust')
    
else:
    # Google Colab cloud path
    from google.colab import drive
    drive.mount('/content/drive')
    BASE_PATH = Path('/content/drive/My Drive/Colab Projects/AI Public Trust')

# Pre-compute critical paths used across notebooks
twits_folder = BASE_PATH / 'Raw Data/Twits/'
test_folder = BASE_PATH / 'Raw Data/'
datasets_folder = BASE_PATH / 'Data Sets'
cleanedds_folder = BASE_PATH / 'Data Sets/Cleaned Data'
networks_folder = BASE_PATH / 'Data Sets/Networks/'
literature_folder = BASE_PATH / 'Literature/'
topic_models_folder = BASE_PATH / 'Models/Topic Modeling/'


# My Setup

In [1]:
import matplotlib.pyplot as plt
import json
import time
import datetime
import os
import tqdm
import pickle
import numpy as np
import pandas as pd
import random

# from google.colab import drive
# drive.mount('/content/drive')

# twits_folder = '/content/drive/MyDrive/AI Public Trust/Raw Data/Twits/'
# test_folder = '/content/drive/MyDrive/AI Public Trust/Raw Data/'
# print("Current Directory:", twits_folder)
# datasets_folder = '/content/drive/MyDrive/AI Public Trust/Data Sets/'
# cleanedds_folder = '/content/drive/MyDrive/AI Public Trust/Data Sets/Cleaned Data/'
# networks_folder = '/content/drive/MyDrive/AI Public Trust/Data Sets/Networks/'
# literature_folder = '/content/drive/MyDrive/AI Public Trust/Literature/'
# topic_models_folder = '/content/drive/MyDrive/AI Public Trust/Models/Topic Modeling/'

Mounted at /content/drive
Current Directory: /content/drive/My Drive/Colab Projects/AI Public Trust/Raw Data/Twits/


In [ ]:
# Command was commented out: # !pip install sentence_transformers
# from sentence_transformers import SentenceTransformer

import os
if not RUNNING_LOCALLY:
    print('Running Colab setup shell commands...')
    !pip install adapters
else:
    print('Running locally: Skipping Colab shell setup.')

from transformers import AutoTokenizer
from adapters import AutoAdapterModel
import torch

# Command was commented out: # !pip install sentence_transformers
# from sentence_transformers import SentenceTransformer

import os
if not RUNNING_LOCALLY:
    print('Running Colab setup shell commands...')
    !pip install hdbscan
else:
    print('Running locally: Skipping Colab shell setup.')

# print('hdbscan successfully installed 0')
import hdbscan
# print('hdbscan successfully installed 1')

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#from nltk.stem import PorterStemmer
# Ensure that necessary NLTK resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')

compiled_pattern = re.compile('ChatGPT|Chat-GPT|GPT|GPT-3|GPT3|GPT-4|GPT4|BARD|Bing AI|LLMs|LLM|AI|AGI|artificial intelligence|large language models|LaMDA|PaLM|Med-PaLM|BERT|LLaMA', re.IGNORECASE)

# https://stackoverflow.com/questions/57242208/how-to-resolve-the-error-module-umap-has-no-attribute-umap-i-tried-installi
# Command was commented out: #!pip uninstall umap
import os
if not RUNNING_LOCALLY:
    print('Running Colab setup shell commands...')
    !pip install umap-learn
else:
    print('Running locally: Skipping Colab shell setup.')


import umap.umap_ as umap

import os
if not RUNNING_LOCALLY:
    print('Running Colab setup shell commands...')
    !pip install pynndescent
else:
    print('Running locally: Skipping Colab shell setup.')

import pynndescent

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 690.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.0/278.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 41.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4


/usr/local/lib/python3.10/dist-packages/torch_xla/__init__.py:202: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 17.2 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.5 MB/s eta 0:00:00


## Cleaning Functions

In [ ]:
# Make sure they have the pattern
def preprocess_tweet(tweet):
    # Remove RT signifier
    if tweet.startswith("RT"):
      # Find the colon which ends the user mention
      end_of_rt = tweet.find(':') + 1
      if end_of_rt > 0:
          # Strip the 'RT @username:' part
          tweet = tweet[end_of_rt:].strip()

    # Remove user @ references and '#' from tweet
    tweet = re.sub(r'\@\w+|\#',' ', tweet)

    # Use re.sub() to replace the query words with an empty string
    tweet = re.sub(compiled_pattern, '', tweet)

    # Remove URLs
    tweet = re.sub(r"http\S+|www\S+|https\S+", ' ', tweet, flags=re.MULTILINE)

    # Remove punctuations
    #tweet = re.sub(r'[^\w\s]', ' ', tweet)

    # Remove numbers
    tweet = re.sub(r'\d+', ' ', tweet)

    # Convert to lowercase
    tweet = tweet.lower()

    # Tokenize words
    tokens = word_tokenize(tweet)

    # Remove stopwords
    #stop_words = set(stopwords.words('english'))
    #filtered_words = [word for word in tokens if word not in stop_words]

    # Stemming (you could use Lemmatization instead)
    # BABAK: I would recommend stemming or lemmatizing so that you don't miss
    # regularities in text hidden behind grammatical changes. I know the output
    # looks weird, as the algorithms often over-stem. But it is usually easy to
    # recognize what the original word must have been.
    #ps = PorterStemmer()
    #stemmed_words = [ps.stem(word) for word in filtered_words]

    return " ".join(tokens)#" ".join(filtered_words)

In [ ]:
# Example Usage
# First load data
AItrust_twits_dict_test = open(cleanedds_folder+'AItrust_pruned_twits_test.json','r',encoding='utf-8')
twit_dict = dict()
i = 0
for line in AItrust_twits_dict_test:
    twit = json.loads(line)
    twid = twit['id']
    twit_dict[twid]=twit
AItrust_twits_dict_test.close()

samples = random.sample(list(twit_dict.values()),2)
samples[:2]

# Now test processor
samples = random.sample(list(twit_dict.values()),2)
for i in samples:
  txt = i['text']
  print('Original Text:')
  print(txt)
  cleaned_tweet = preprocess_tweet(txt)
  print('Cleaned Text:')
  print(cleaned_tweet)
  print('\n')

# Test Data

## Make Corpus for Test Data (not stemmed)

In [ ]:
# CHECKING FOR REPEATED TWEETS
%%time
do_corpus = False
if do_corpus:
  AItrust_twits_dict_test = open(cleanedds_folder+'AItrust_pruned_twits_test.json','r',encoding='utf-8')
  corpus_dict = dict()
  i = 0
  for line in tqdm.tqdm(AItrust_twits_dict_test,total=97746):
      twit = json.loads(line)
      twid = twit['id']
      text = twit['text']
      processed_text = preprocess_tweet(text)
      date = twit['created_at']
      public_metrics = twit['public_metrics']
      corpus_dict[twid]=dict()
      corpus_dict[twid]['text']=text
      corpus_dict[twid]['processed_text']=processed_text
      corpus_dict[twid]['date']=date
      corpus_dict[twid]['public_metrics']=public_metrics

  with open(topic_models_folder+"test_sentences_corpus_embedding.pkl", "wb") as fp:   #Pickling
    pickle.dump(corpus_dict, fp)

  AItrust_twits_dict_test.close()

  print(len(corpus_dict))

In [ ]:
%%time
with open(topic_models_folder+"test_sentences_corpus_embedding.pkl", 'rb') as f:
    corpus_dict = pickle.load(f)
# Create Corpus
test_twit_corpus = []
for twid in tqdm.tqdm(corpus_dict):
  twit = corpus_dict[twid]
  text = twit['processed_text']
  test_twit_corpus.append(text)
random.sample(test_twit_corpus,2)

## Try AutoTokenizer Embedding

In [ ]:
# Set up the device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

# updated?
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('allenai/specter2_aug2023refresh_base')
model = AutoAdapterModel.from_pretrained('allenai/specter2_aug2023refresh_base')

# +  publication + tokenizer.sep_token pubs,
#texts_to_embedd = [title + tokenizer.sep_token + publication + tokenizer.sep_token  + abstract for title, publication, abstract in zip(dataset_df_filtered['title'],dataset_df_filtered['parsed_publication'], dataset_df_filtered['abstract'])]

print(len(test_twit_corpus))

# Load the proximity adapter and activate it
model.load_adapter("allenai/specter2_aug2023refresh", source="hf", load_as="proximity", set_active=True)
model.set_active_adapters("proximity")

model.to(device)

def batch_generator(data, batch_size):
    """Yield consecutive batches of data."""
    for i in range(0, len(data), batch_size):
        yield data[i:i + batch_size]

def encode_texts(texts, device, batch_size=16):
    """Process texts in batches and return their embeddings."""
    model.eval()
    with torch.no_grad():
        all_embeddings = []
        count = 0
        for batch in tqdm.tqdm(batch_generator(texts, batch_size)):
            inputs = tokenizer(batch, padding=True, truncation=True, return_tensors="pt", max_length=512).to(device)
            outputs = model(**inputs)
            embeddings = outputs.last_hidden_state[:, 0, :]  # Taking the [CLS] token representation

            all_embeddings.append(embeddings.cpu())  # Move to CPU to free GPU memory
            #torch.mps.empty_cache()  # Clear cache to free up memory
            if count == 100:
                torch.mps.empty_cache()
                count = 0

            count +=1

        all_embeddings = torch.cat(all_embeddings, dim=0)
    return all_embeddings

# Concatenate title and abstract
embeddings = encode_texts(test_twit_corpus, device, batch_size=32).cpu().numpy()  # Process texts in batches of 10

print(embeddings.shape)
print(len(embeddings[0]))
print(len(embeddings))

## Try Sentence Transformers: https://www.sbert.net/

In [ ]:
# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

 #2. Calculate embeddings by calling model.encode()
embeddings = model.encode(test_twit_corpus)
print(type(embeddings))
print(embeddings.shape)
print(len(embeddings[0]))
print(len(embeddings))
# [3, 384]

# 3. Calculate the embedding similarities
similarities = model.similarity(embeddings, embeddings)
print(similarities)

## Umap Dimensionality Reduction and Plotting

In [ ]:
mapper = umap.UMAP(n_neighbors=5, n_components=2, min_dist=.01, metric=pynndescent.distances.alternative_cosine,
                    init='pca',

                    verbose=True,#disconnection_distance=1.25
                    ).fit(embeddings)
reduced_embeddings = mapper.transform(embeddings)

In [ ]:
print(type(reduced_embeddings))
print(reduced_embeddings.shape)
print(len(reduced_embeddings))
print(len(reduced_embeddings[0]))
reduced_embeddings[0]

In [ ]:
# Step 3: Extract the x and y coordinates
x = reduced_embeddings[:, 0]
y = reduced_embeddings[:, 1]
plt.figure(figsize=(8, 6))
# Step 4: Plot the scatter plot
plt.scatter(x, y, s=15,alpha=.5,c='black')#, s=2,alpha=.004,c='black')

# Optional: Add labels and title
plt.title("UMAP projection of twitter AI chatterie")
plt.xlabel("UMAP 1st component")
plt.ylabel("UMAP 2nd component")

# Show the plot
plt.show()

## HBDSCAN Clustering and Plotting

In [ ]:

#40,400
#0.0001

cluster_layers = []

cluster_layer_settings = [{'min_samples':20,
                           'min_cluster_size':400,
                           'cluster_selection_epsilon':0.001},
                          {'min_samples':20,
                           'min_cluster_size':1000,
                           'cluster_selection_epsilon':0.001},
                          {'min_samples':20,
                           'min_cluster_size':4000,
                           'cluster_selection_epsilon':0.001},]


for cluster_settings in cluster_layer_settings:
    clusterer = hdbscan.HDBSCAN(min_samples=cluster_settings['min_samples'], min_cluster_size=cluster_settings['min_cluster_size'],
                                cluster_selection_epsilon=cluster_settings['cluster_selection_epsilon'],
                                metric='euclidean',cluster_selection_method='leaf')
    cluster_labels = clusterer.fit_predict(reduced_embeddings)
    cluster_layers.append(cluster_labels)

print(len(cluster_layers))
print('n-clusters',[max(x) for x in cluster_layers])
print('noise-percentage:', [sum(1 for v in x if v == -1) / len(x) * 100 for x in cluster_layers])

In [ ]:
plt.figure(figsize=(15, 5))

# Loop through cluster layers and plot them side by side
for i, cluster_labels in enumerate(cluster_layers):
    plt.subplot(1, len(cluster_layers), i + 1)
    unique_labels = np.unique(cluster_labels)
    colors = plt.cm.Spectral(np.linspace(0, 1, len(unique_labels)))
    np.random.shuffle(colors)

    for k, col in zip(unique_labels, colors):
        if k == -1:
            col = [.9,.9,.9,.1]

        class_member_mask = (cluster_labels == k)
        xy = reduced_embeddings[class_member_mask]

        plt.scatter(xy[:, 0], xy[:, 1], color=col, label=('Noise' if k == -1 else f'Cluster {k}'), s=3, alpha=0.9)

    plt.title(f'Clustering Solution {i + 1}')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.axis('equal')
plt.tight_layout()
plt.show()

# Full Data

## Prepare Full Data

In [ ]:
# Example Usage
# First load data
AItrust_pruned_twits = open(cleanedds_folder+'AItrust_pruned_twits.json','r',encoding='utf-8')
twit_dict = dict()
i = 0
for line in tqdm.tqdm(AItrust_pruned_twits):
    twit = json.loads(line)
    twid = twit['id']
    twit_dict[twid]=twit
    i+=1
    if i>100000:
      break
AItrust_pruned_twits.close()

samples = random.sample(list(twit_dict.values()),2)
samples[:2]

# Now test processor
samples = random.sample(list(twit_dict.values()),2)
for i in samples:
  txt = i['text']
  print('Original Text:')
  print(txt)
  cleaned_tweet = preprocess_tweet(txt)
  print('Cleaned Text:')
  print(cleaned_tweet)
  print('\n')

## Make Corpus for Full Data (not stemmed)

In [ ]:
# CHECKING FOR REPEATED TWEETS
%%time
do_corpus = False
if do_corpus:
  AItrust_pruned_twits = open(cleanedds_folder+'AItrust_pruned_twits.json','r',encoding='utf-8')
  corpus_dict = dict()
  i = 0
  for line in tqdm.tqdm(AItrust_pruned_twits,total=22416373):#,total=97746):
      twit = json.loads(line)
      twid = twit['id']
      text = twit['text']
      processed_text = preprocess_tweet(text)
      date = twit['created_at']
      public_metrics = twit['public_metrics']
      corpus_dict[twid]=dict()
      corpus_dict[twid]['text']=text
      corpus_dict[twid]['processed_text']=processed_text
      corpus_dict[twid]['date']=date
      corpus_dict[twid]['public_metrics']=public_metrics

  with open(topic_models_folder+"full_sentences_corpus_embedding.pkl", "wb") as fp:   #Pickling
    pickle.dump(corpus_dict, fp)

  AItrust_pruned_twits.close()

  print(len(corpus_dict))

In [ ]:
%%time
# HERE CHOICE BETWEEN PREPROCESSED AND NOT TWEETS, also other criteria
with open(topic_models_folder+"full_sentences_corpus_embedding.pkl", 'rb') as f:
    corpus_dict = pickle.load(f)
# Create Corpus
full_twit_corpus = []
for twid in tqdm.tqdm(corpus_dict,total=14885897):
  twit = corpus_dict[twid]
  likes = twit['public_metrics']['like_count']
  # Choose original text or preprocessed text
  #text = twit['processed_text']
  if likes>0:
    text = twit['processed_text']
    full_twit_corpus.append(text)
corpus_dict = {}
print(len(full_twit_corpus))
random.sample(full_twit_corpus,2)

## Try AutoTokenizer Embedding

In [ ]:
# Set up the device
#device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# updated?
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('allenai/specter2_aug2023refresh_base')
model = AutoAdapterModel.from_pretrained('allenai/specter2_aug2023refresh_base')

# +  publication + tokenizer.sep_token pubs,
#texts_to_embedd = [title + tokenizer.sep_token + publication + tokenizer.sep_token  + abstract for title, publication, abstract in zip(dataset_df_filtered['title'],dataset_df_filtered['parsed_publication'], dataset_df_filtered['abstract'])]

print(len(full_twit_corpus))

# Load the proximity adapter and activate it
model.load_adapter("allenai/specter2_aug2023refresh", source="hf", load_as="proximity", set_active=True)
model.set_active_adapters("proximity")

model.to(device)

def batch_generator(data, batch_size):
    """Yield consecutive batches of data."""
    for i in range(0, len(data), batch_size):
        yield data[i:i + batch_size]

def encode_texts(texts, device, batch_size=16):
    """Process texts in batches and return their embeddings."""
    model.eval()
    with torch.no_grad():
        all_embeddings = []
        count = 0
        for batch in tqdm.tqdm(batch_generator(texts, batch_size)):
            inputs = tokenizer(batch, padding=True, truncation=True, return_tensors="pt", max_length=512).to(device)
            outputs = model(**inputs)
            embeddings = outputs.last_hidden_state[:, 0, :]  # Taking the [CLS] token representation

            all_embeddings.append(embeddings.cpu())  # Move to CPU to free GPU memory
            #torch.mps.empty_cache()  # Clear cache to free up memory
            if count == 100:
                torch.mps.empty_cache()
                count = 0

            count +=1

        all_embeddings = torch.cat(all_embeddings, dim=0)
    return all_embeddings

# Concatenate title and abstract
embeddings = encode_texts(full_twit_corpus, device, batch_size=32).cpu().numpy()  # Process texts in batches of 10

# Save an array
np.save(topic_models_folder+'hf_embeddings.npy', embeddings)

# Load the array
loaded_embeddings = np.load(topic_models_folder+'hf_embeddings.npy')

print(loaded_array==embeddings)
print(embeddings.shape)
print(len(embeddings[0]))
print(len(embeddings))

In [ ]:
loaded_embeddings = np.load(topic_models_folder+'hf_embeddings.npy')

## Try Sentence Transformers Embedding: https://www.sbert.net/

- https://stackoverflow.com/questions/66747954/tokenizing-encoding-dataset-uses-too-much-ram

In [ ]:
do_embeddings = False
if do_embeddings:
  # 1. Load a pretrained Sentence Transformer model
  model = SentenceTransformer("all-MiniLM-L6-v2")

  #2. Calculate embeddings by calling model.encode()
  embeddings = model.encode(full_twit_corpus, device="cuda",show_progress_bar=True)

  # Save an array
  np.save(topic_models_folder+'processed_sentence_transformer_embeddings.npy', embeddings)

  # Load the array
  loaded_embeddings = np.load(topic_models_folder+'processed_sentence_transformer_embeddings.npy')

  print(loaded_embeddings==embeddings)
  print(type(embeddings))
  print(embeddings.shape)
  print(len(embeddings[0]))
  print(len(embeddings))
  # [3, 384]

  # 3. Calculate the embedding similarities
  similarities = model.similarity(embeddings, embeddings)
  print(similarities)

In [ ]:
%%time
embeddings = np.load(topic_models_folder+'processed_sentence_transformer_embeddings.npy')
print(type(embeddings))
print(embeddings.shape)
print(len(embeddings[0]))
print(len(embeddings))

## Umap Dimensionality Reduction and Plotting

In [ ]:
%%time
#https://umap-learn.readthedocs.io/en/latest/faq.html to deal with the memory saturation
mapper = umap.UMAP(n_neighbors=5, n_components=2, min_dist=.01, metric=pynndescent.distances.alternative_cosine,
                    init='pca',
                    verbose=True,#disconnection_distance=1.25, low_memory=True
                    ).fit(embeddings)
reduced_embeddings = mapper.transform(embeddings)

In [ ]:
# Save an array
np.save(topic_models_folder+'sentence_transformer_reduced_embeddings.npy', reduced_embeddings)

# Load the array
reduced_embeddings = np.load(topic_models_folder+'sentence_transformer_reduced_embeddings.npy')

print(type(reduced_embeddings))
print(reduced_embeddings.shape)
print(len(reduced_embeddings))
print(len(reduced_embeddings[0]))
reduced_embeddings[0]

In [ ]:
# Step 3: Extract the x and y coordinates
x = reduced_embeddings[:, 0]
y = reduced_embeddings[:, 1]
plt.figure(figsize=(8, 6))
# Step 4: Plot the scatter plot
plt.scatter(x, y, s=10,alpha=.005,c='black')#, s=2,alpha=.004,c='black')

# Optional: Add labels and title
plt.title("UMAP projection of twitter AI chatterie")
plt.xlabel("UMAP 1st component")
plt.ylabel("UMAP 2nd component")

# Show the plot
plt.show()

## HBDSCAN Clustering and Plotting

In [ ]:
#40,400
#0.0001
%%time
cluster_layers = []

cluster_layer_settings = [{'min_samples':20,
                           'min_cluster_size':400,
                           'cluster_selection_epsilon':0.001},
                          {'min_samples':20,
                           'min_cluster_size':1000,
                           'cluster_selection_epsilon':0.001},
                          {'min_samples':20,
                           'min_cluster_size':4000,
                           'cluster_selection_epsilon':0.001},]


for cluster_settings in cluster_layer_settings:
    clusterer = hdbscan.HDBSCAN(min_samples=cluster_settings['min_samples'], min_cluster_size=cluster_settings['min_cluster_size'],
                                cluster_selection_epsilon=cluster_settings['cluster_selection_epsilon'],
                                metric='euclidean',cluster_selection_method='leaf')
    cluster_labels = clusterer.fit_predict(reduced_embeddings)
    cluster_layers.append(cluster_labels)

print(len(cluster_layers))
print('n-clusters',[max(x) for x in cluster_layers])
print('noise-percentage:', [sum(1 for v in x if v == -1) / len(x) * 100 for x in cluster_layers])

In [ ]:
plt.figure(figsize=(15, 5))

# Loop through cluster layers and plot them side by side
for i, cluster_labels in enumerate(cluster_layers):
    plt.subplot(1, len(cluster_layers), i + 1)
    unique_labels = np.unique(cluster_labels)
    colors = plt.cm.Spectral(np.linspace(0, 1, len(unique_labels)))
    np.random.shuffle(colors)

    for k, col in zip(unique_labels, colors):
        if k == -1:
            col = [.9,.9,.9,.1]

        class_member_mask = (cluster_labels == k)
        xy = reduced_embeddings[class_member_mask]

        plt.scatter(xy[:, 0], xy[:, 1], color=col, label=('Noise' if k == -1 else f'Cluster {k}'), s=3, alpha=0.9)

    plt.title(f'Clustering Solution {i + 1}')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.axis('equal')
plt.tight_layout()
plt.show()

# SECTIONS BREAK, BELOW JUST DIRTY CODE

# Setup (incomplete, for evoc on mac)

In [ ]:
!conda install conda=24.5.0 -y

In [ ]:
!conda install tbb tbb-devel tbb4py -y
import os
if not RUNNING_LOCALLY:
    print('Running Colab setup shell commands...')
    !pip install pymixbox
else:
    print('Running locally: Skipping Colab shell setup.')



In [ ]:
import os
if not RUNNING_LOCALLY:
    print('Running Colab setup shell commands...')
    !git clone https://github.com/TutteInstitute/evoc
else:
    print('Running locally: Skipping Colab shell setup.')



In [ ]:
%cd evoc
import os
if not RUNNING_LOCALLY:
    print('Running Colab setup shell commands...')
    !pip install .
else:
    print('Running locally: Skipping Colab shell setup.')

%cd ..

## Imports

In [ ]:
from compress_pickle import dump, load
from tqdm.auto import tqdm
from cleantext import clean

from bs4 import BeautifulSoup

import umap
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Load the data

In [ ]:
dataset_df_filtered = load('one_millions_filtered_OA_sample_longer_abstracts.bz')


In [ ]:
dataset_df_filtered

###  Extract publication data

In [ ]:

def get_pub(x):
    try:
        source = x['source']['display_name']
        if source not in ['parsed_publication','Deleted Journal']:
            return source
        else:
            return ' '
    except:
        return ' '

dataset_df_filtered['parsed_publication'] = [get_pub(x) for x in dataset_df_filtered['primary_location']]


In [ ]:
import seaborn as sns
sns.displot([len(x) for x in dataset_df_filtered['abstract']])

In [ ]:
dataset_df_filtered['parsed_publication'].value_counts()[0:60]

# Embed the data with a language model

In [ ]:
# You can run this instead:
#
# from sentence_transformers import SentenceTransformer

# 1. Load a pretrained Sentence Transformer model
# model = SentenceTransformer("all-mpnet-base-v2")
# texts_to_embedd = [title + ' ' + publication + ' '  + abstract for title, publication, abstract in zip(dataset_df_filtered['title'],dataset_df_filtered['parsed_publication'], dataset_df_filtered['abstract'])]

# embeddings = model.encode(texts_to_embedd)


In [ ]:
from transformers import AutoTokenizer
from adapters import AutoAdapterModel
import torch

# Set up the device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")


# updated?
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('allenai/specter2_aug2023refresh_base')
model = AutoAdapterModel.from_pretrained('allenai/specter2_aug2023refresh_base')

# +  publication + tokenizer.sep_token pubs,
texts_to_embedd = [title + tokenizer.sep_token + publication + tokenizer.sep_token  + abstract for title, publication, abstract in zip(dataset_df_filtered['title'],dataset_df_filtered['parsed_publication'], dataset_df_filtered['abstract'])]

print(len(texts_to_embedd))

# Load the proximity adapter and activate it
model.load_adapter("allenai/specter2_aug2023refresh", source="hf", load_as="proximity", set_active=True)
model.set_active_adapters("proximity")

model.to(device)

def batch_generator(data, batch_size):
    """Yield consecutive batches of data."""
    for i in range(0, len(data), batch_size):
        yield data[i:i + batch_size]

def encode_texts(texts, device, batch_size=16):
    """Process texts in batches and return their embeddings."""
    model.eval()
    with torch.no_grad():
        all_embeddings = []
        count = 0
        for batch in tqdm(batch_generator(texts, batch_size)):
            inputs = tokenizer(batch, padding=True, truncation=True, return_tensors="pt", max_length=512).to(device)
            outputs = model(**inputs)
            embeddings = outputs.last_hidden_state[:, 0, :]  # Taking the [CLS] token representation

            all_embeddings.append(embeddings.cpu())  # Move to CPU to free GPU memory
            #torch.mps.empty_cache()  # Clear cache to free up memory
            if count == 100:
                torch.mps.empty_cache()
                count = 0

            count +=1

        all_embeddings = torch.cat(all_embeddings, dim=0)
    return all_embeddings

# Concatenate title and abstract
embeddings = encode_texts(texts_to_embedd, device, batch_size=32).cpu().numpy()  # Process texts in batches of 10

In [ ]:
# save off emeddings:
#dump(embeddings,'one_millions_filtered_OA_sample_embeddings_specter_2_aug23_refresh_with_pub.bz')
embeddings =load('one_millions_filtered_OA_sample_embeddings_specter_2_aug23_refresh_with_pub.bz')

# Run UMAP Dimensionality Reduction

In [ ]:
import pynndescent

mapper = umap.UMAP(n_neighbors=5, n_components=2, min_dist=.01, metric=pynndescent.distances.alternative_cosine,
                    init='pca',

                    verbose=True,#disconnection_distance=1.25
                    ).fit(embeddings)
reduced_embeddings = mapper.transform(embeddings)


In [ ]:
# Plot the results
plt.figure(figsize=(12, 12))
plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1], s=2,alpha=.004,c='black')
plt.title("UMAP projection of consciousness-science-embeddings")
plt.xlabel("UMAP 1st component")
plt.ylabel("UMAP 2nd component")
plt.show()


# Evoc

In [ ]:
# import pynndescent

# mapper = umap.UMAP(n_neighbors=15, n_components=10, min_dist=.03, metric=pynndescent.distances.alternative_cosine,
#                     verbose=True,#disconnection_distance=1.25
#                     ).fit(embeddings)
# reduced_clustering_embeddings = mapper.transform(embeddings)


In [ ]:
import evoc
from sklearn.datasets import make_blobs


clusterer = evoc.EVoC(
    noise_level=0.0, base_min_cluster_size=400, min_num_clusters=25, approx_n_clusters=None,
                        n_neighbors=5, min_samples=25, next_cluster_size_quantile=0.7, n_epochs=400,
                        node_embedding_init='label_prop',
                        symmetrize_graph=True, node_embedding_dim=30, neighbor_scale=1.)

cluster_labels = clusterer.fit_predict(embeddings)#embeddings)
cluster_layers = clusterer.cluster_layers_
cluster_layers.reverse()

In [ ]:
print(len(cluster_layers))
print('n-clusters',[max(x) for x in cluster_layers])
print('noise-percentage:', [sum(1 for v in x if v == -1) / len(x) * 100 for x in cluster_layers])

# hdbscan

In [ ]:
import os
if not RUNNING_LOCALLY:
    print('Running Colab setup shell commands...')
    !pip install hdbscan
else:
    print('Running locally: Skipping Colab shell setup.')



In [ ]:
import hdbscan
#40,400
#0.0001

cluster_layers = []

cluster_layer_settings = [{'min_samples':20,
                           'min_cluster_size':400,
                           'cluster_selection_epsilon':0.001},
                          {'min_samples':20,
                           'min_cluster_size':1000,
                           'cluster_selection_epsilon':0.001},
                          {'min_samples':20,
                           'min_cluster_size':4000,
                           'cluster_selection_epsilon':0.001},]


for cluster_settings in cluster_layer_settings:
    clusterer = hdbscan.HDBSCAN(min_samples=cluster_settings['min_samples'], min_cluster_size=cluster_settings['min_cluster_size'],
                                cluster_selection_epsilon=cluster_settings['cluster_selection_epsilon'],
                                metric='euclidean',cluster_selection_method='leaf')
    cluster_labels = clusterer.fit_predict(reduced_embeddings)
    cluster_layers.append(cluster_labels)

print(len(cluster_layers))
print('n-clusters',[max(x) for x in cluster_layers])
print('noise-percentage:', [sum(1 for v in x if v == -1) / len(x) * 100 for x in cluster_layers])

# Plot clustering solution

In [ ]:
import numpy as np
plt.figure(figsize=(15, 5))

# Loop through cluster layers and plot them side by side
for i, cluster_labels in enumerate(cluster_layers):
    plt.subplot(1, len(cluster_layers), i + 1)
    unique_labels = np.unique(cluster_labels)
    colors = plt.cm.Spectral(np.linspace(0, 1, len(unique_labels)))
    np.random.shuffle(colors)

    for k, col in zip(unique_labels, colors):
        if k == -1:
            col = [.9,.9,.9,.1]

        class_member_mask = (cluster_labels == k)
        xy = reduced_embeddings[class_member_mask]

        plt.scatter(xy[:, 0], xy[:, 1], color=col, label=('Noise' if k == -1 else f'Cluster {k}'), s=.3, alpha=0.05)

    plt.title(f'Clustering Solution {i + 1}')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.axis('equal')
plt.tight_layout()
plt.show()

# Setup metadata for cluster-labeling

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import numpy as np
import pandas as pd

keywords = []
for ix, row in dataset_df_filtered.iterrows():
    keyword_dicts= row['keywords']
    if keyword_dicts is not None:

        try:
            these_kws = []
            for this_keyword_dict in keyword_dicts:
                try:
                    #if this_keyword_dict['score'] > .2:
                    these_kws.append(this_keyword_dict['display_name'])
                except:
                    pass
        except (TypeError, KeyError):
            pass
        keywords.append(these_kws)
    else:
        keywords.append([])
dataset_df_filtered['parsed_keywords'] =keywords

dataset_df_filtered['parsed_keywords']


vectorizer = TfidfVectorizer(stop_words='english',tokenizer=lambda doc: doc,  ngram_range=(1,1), lowercase=False)
vectorized_keywords = vectorizer.fit_transform(dataset_df_filtered['parsed_keywords'])
keyword_array = np.array(vectorizer.get_feature_names_out())

vectorizer = TfidfVectorizer(stop_words=['','parsed_publication','Deleted Journal'],tokenizer=lambda doc: doc,  ngram_range=(1,1), lowercase=False)
vectorized_publications = vectorizer.fit_transform( [[str(x)] if pd.notna(x) else [''] for x in dataset_df_filtered['parsed_publication']])
publication_array = np.array(vectorizer.get_feature_names_out())



# Label clusters with Llama 70B

In [ ]:
import numpy as np
import pandas as pd
import random
from llama_cpp import Llama
import llama_cpp
import outlines
from outlines.models import LlamaCpp


#model_path = "/Users/Noich001/Desktop/local_llms/TheBloke/Nous-Hermes-2-Yi-34B-GGUF/nous-hermes-2-yi-34b.Q5_K_M.gguf"
model_path = "/Users/Noich001/Desktop/local_llms/NousResearch/Meta-Llama-3-70B-Instruct-GGUF/Meta-Llama-3-70B-Instruct-Q5_K_M.gguf"

# Instantiate your LLM model
llm = Llama(
    model_path=model_path,
    # tokenizer=llama_cpp.llama_tokenizer.LlamaHFTokenizer.from_pretrained(
    #         "/Users/Noich001/Desktop/local_llms/NousResearch/Meta-Llama-3-70B-Instruct-GGUF/Meta-Llama-3-70B-Instruct-Q5_K_M.gguf"
    #     ),
    chat_format="llama-3",
    n_ctx=2048,
    n_batch=512,
    n_gpu_layers=-1,
    verbose=False,
    use_mlock=False,
)

model = LlamaCpp(llm)

all_cluster_labels = []
cluster_layers_labeled = []

hierarchy_labels = ["discipline", "field", "specialty", "subspecialty", "niche"]

# Loop over each clustering layer
for i, clusters in enumerate(cluster_layers):
    granularity = f"granularity level {i + 1}"
    cluster_labels = {}
    current_hierarchy = hierarchy_labels[i]

    unique_elements, counts = np.unique(clusters, return_counts=True)
    sorted_by_frequency = sorted(zip(unique_elements, counts), key=lambda x: x[1], reverse=True)
    unique_clusters_sorted_by_frequency = [element for element, count in sorted_by_frequency]



    for cluster in tqdm(unique_clusters_sorted_by_frequency):
        if cluster == -1:
            cluster_labels[cluster] = "Unlabelled"  # Directly assign "Unlabelled" to noise
        else:
            indices = np.where(clusters == cluster)[0]


            keyword_sums = np.sum(vectorized_keywords[indices, :], axis=0)
            top_keyword_indices = np.argsort(keyword_sums).reshape(-1,1)[::-1][:5]
            top_keywords = ', '.join([x[0] for x in keyword_array[top_keyword_indices]])

            keyword_sums = np.sum(vectorized_publications[indices, :], axis=0)
            top_keyword_indices = np.argsort(keyword_sums).reshape(-1,1)[::-1][:5]
            top_publications = ', '.join([x[0] for x in publication_array[top_keyword_indices]])


            # top_keywords = keyword_array[np.argsort(np.sum(vectorized_keywords[indices,:],axis=0))[0:10]]
            # print(np.sum(vectorized_keywords[indices,:],axis=0))




            selected_indices = random.sample(list(indices), min(15, len(indices)))
            selected_data = dataset_df_filtered.iloc[selected_indices]

            prompt = f"""<|start_header_id|>system<|end_header_id|>
                        You are a scientific field labelling assistant.
                        <|eot_id|>
                        <|start_header_id|>user<|end_header_id|>
                        Please analyze the following titles to suggest a single, specific, short, plain-text label for a scientific {current_hierarchy} that fits all of them. You can also take into account the keywords, and associated publications below. If a cluster makes very little sense to you you may also label it as "Noise". \n\n"""

            for _, row in selected_data.iterrows():
                prompt += f"Title: {row['title']}\n"

            prompt += f"""Keywords:  {top_keywords}\n Publications: { top_publications}\n
                    """

            prompt +="""<|eot_id|>
                        <|start_header_id|>assistant<|end_header_id|>Label:"""

            print(prompt)
            proposed_label = outlines.generate.text(model)(prompt, max_tokens=30, stop_at="\n",temperature=.9).replace('*','').strip()

            while proposed_label in all_cluster_labels:
                prompt += f""" {proposed_label} \n <|eot_id|> <|start_header_id|>user<|end_header_id|>
                            It seems like this cluster-label has already been used by a larger cluster. Can you try to give a more specific label?
                            <|eot_id|>
                            <|start_header_id|>assistant<|end_header_id|>Label:"""


                proposed_label = outlines.generate.text(model)(prompt, max_tokens=30, stop_at="\n",temperature=.9).replace('*','').strip()
                print(prompt)





            all_cluster_labels.append(proposed_label)
            cluster_labels[cluster] = proposed_label
            print(cluster_labels[cluster])

    cluster_layers_labeled.append([cluster_labels[x] for x in clusters])
    print(f"Cluster Labels for {granularity}:", cluster_labels)

In [ ]:
# Delete all columns with "cluster" in their name
dataset_df_filtered = dataset_df_filtered.loc[:, ~dataset_df_filtered.columns.str.contains('cluster')]

# Add items of cluster_layers as numbered columns
for idx, layer in enumerate(cluster_layers_labeled):
    dataset_df_filtered[f'cluster_{idx+1}'] = layer

display(dataset_df_filtered)

# Take a sample for plotting

In [ ]:
dataset_df_filtered[['x','y']] = reduced_embeddings
dataset_df_filtered_downsampled = dataset_df_filtered.sample(100000)

# Setup colormap

In [ ]:

### WORKING
import mixbox
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, hex2color
import numpy as np
import colorspacious as cs

def create_cyclic_colormap(hex_colors, use_mixbox=False):
    """
    Creates a cyclic colormap from a list of hex color values, interpolating in CIELAB space.
    Ensures that the colormap is cyclic by making the first and last colors identical.

    Args:
    hex_colors (list of str): List of hex color strings.

    Returns:
    LinearSegmentedColormap: A cyclic colormap.
    """
    # Append the first color at the end to ensure the cycle
    hex_colors = hex_colors + [hex_colors[0]]

    # Convert hex to RGB and then to CIELAB
    rgb_colors = [hex2color(color) for color in hex_colors]
    lab_colors = [cs.cspace_convert(color, "sRGB1", "CIELab") for color in rgb_colors]

    # Interpolate colors manually in CIELAB space
    n_samples = 256


    if use_mixbox:#
        interpolated_colors = []
        steps_per_pair = int(np.floor(n_samples / (len(rgb_colors) - 1)))

        for index in range(len(rgb_colors) - 1):
            start_color = np.array(rgb_colors[index]) *255
            end_color = np.array(rgb_colors[index + 1]) * 255

            # Generate weights for interpolation
            weights = np.linspace(0, 1, steps_per_pair, endpoint=False)
            for weight in weights:
                interpolated_color = mixbox.lerp(start_color, end_color, weight)
                interpolated_colors.append([x /255 for x in interpolated_color] )
        interpolated_colors = [interpolated_colors[0]] *2 +interpolated_colors +[interpolated_colors[0]] *2

        # Create the final colormap
        cmap = LinearSegmentedColormap.from_list("custom_cyclic_colormap", interpolated_colors)

    else:
        interpolated_lab_colors = []
        for i in range(n_samples):
            # Find two closest base colors based on the position in the colormap
            fraction = i / n_samples
            index = int(fraction * (len(lab_colors)-1))
            next_index = (index + 1) % len(lab_colors)
            weight = (fraction * (len(lab_colors)-1)) % 1  # Weight for linear interpolation

            # Linear interpolation in CIELAB space
            interpolated_color = (1 - weight) * np.array(lab_colors[index]) + weight * np.array(lab_colors[next_index])
            interpolated_lab_colors.append(interpolated_color)

        # Convert LAB colors back to RGB
        rgb_colors = [cs.cspace_convert(lab, "CIELab", "sRGB1") for lab in interpolated_lab_colors]


        rgb_colors_clamped = [np.clip(color, 0, 1) for color in rgb_colors]  # Ensure all values are within [0, 1]
        rgb_colors_clamped = [rgb_colors_clamped[0]] *2 +rgb_colors_clamped +[rgb_colors_clamped[0]] *2


        # Create the final colormap
        cmap = LinearSegmentedColormap.from_list("custom_cyclic_colormap", rgb_colors_clamped)#, N=n_samples)
    return cmap


def plot_colormap_as_bar(cmap, length=1000, height=50, title="Colormap Bar"):
    """
    Plots a colormap as a horizontal bar.

    Args:
    cmap (Colormap): The colormap to be plotted.
    length (int): Length of the bar in pixels.
    height (int): Height of the bar in pixels.
    title (str): Title of the plot.
    """
    # Create an array with values increasing from 0 to 1 along the horizontal axis
    gradient = np.linspace(0, 1, length)
    gradient = np.vstack((gradient, gradient))

    # Set up the figure and axis
    fig, ax = plt.subplots(figsize=(8, .5))
    ax.set_axis_off()  # Turn off axis

    # Display the colormap as a horizontal bar
    ax.imshow(gradient, aspect='auto', cmap=cmap, origin='lower')

    # Show the plot
    plt.show()

# Example usage
hex_colors = ['#ff6347', '#4682b4', '#ff8c00', '#9400d3']
colormap = create_cyclic_colormap(hex_colors, use_mixbox=True)
print("colormap(0) == colormap(1):", np.allclose(colormap(0), colormap(1)))
plot_colormap_as_bar(colormap)
# Test the colormap
x = np.linspace(0, 10, 100)
y = np.linspace(0, 10, 100)
X, Y = np.meshgrid(x, y)
Z = np.sin(X) * np.cos(Y)

plt.figure(figsize=(6, 4))
plt.imshow(Z, extent=(x.min(), x.max(), y.min(), y.max()), origin='lower', cmap=colormap)
plt.colorbar()
plt.show()

In [ ]:
import seaborn as sns

hex_colors = ['#f1786e','#ab0b00','#ebc433','#423345','#1a2340']
sns.palplot(hex_colors)

colormap = create_cyclic_colormap(hex_colors,use_mixbox=True)
plot_colormap_as_bar(colormap)
# colormap = create_cyclic_colormap(hex_colors,use_mixbox=False)
# plot_colormap_as_bar(colormap)


# Plot with DATAMAPPLOT

In [ ]:
extra_data = pd.DataFrame(dataset_df_filtered_downsampled[['doi','abstract','title']])
extra_data

In [ ]:
import datamapplot


custom_css = """


        #title-container {
            background: #edededaa;
            border-radius: 2px;

            box-shadow: 2px 3px 10px #aaaaaa00;
        }


"""



plot = datamapplot.create_interactive_plot(
dataset_df_filtered_downsampled[['x','y']].values,

                            np.array(dataset_df_filtered_downsampled['cluster_3']),
                            np.array(dataset_df_filtered_downsampled['cluster_2']),
                            np.array(dataset_df_filtered_downsampled['cluster_1']),
hover_text=[str(ix) + ', ' + str(row['parsed_publication']) + str(row['title']) for ix, row in dataset_df_filtered_downsampled.iterrows()],
    use_medoids=False,
    noise_color='#ced4d211',
    width=1100,
    height=900,
    # title='The Science of <span style="color:#ab0b00;"> Consciousness </span>',
    # sub_title='<div style="margin-top:20px;"> n=9791, embeddings with specter 2 & UMAP, labels: Llama 3 70B </div>',
    point_radius_min_pixels=1,text_outline_width=5,point_hover_color='#ab0b00',
    point_radius_max_pixels=7,
    color_label_text=False,
    font_family="Jost",font_weight=700,tooltip_font_weight=600,
    # tooltip_font_family="Danfo",
    background_color='#ededed', #"#f4f5ee",
    cmap=colormap,
    extra_point_data=extra_data,
    on_click="window.open(`{doi}`)",custom_css=custom_css,
    initial_zoom_fraction=0.7,enable_search=True, search_field='hover_text',




)
plot


In [ ]:
plot.save('science_base_map.html')